In [255]:
import pandas as pd
import numpy as np
import json


# Importing files

In [256]:
folder = 'tmdb-box-office-prediction'

trainingData = pd.read_csv(f'{folder}/train.csv')
testData = pd.read_csv(f'{folder}/test.csv')


## Dividing into features and label

In [257]:
try:
    label = trainingData['revenue']
    id = trainingData['id']
except:
    pass
dropFields = ['homepage', 'revenue', 'id', 'original_title', 'imdb_id',
              'status', 'title', 'tagline', 'poster_path', 'overview', 'belongs_to_collection']
dropFields2 = ['Keywords', 'cast', 'crew', 'spoken_languages', 'production_countries']
features = trainingData.drop(columns=dropFields + dropFields2)
#print(trainingData.size)

In [258]:
features.keys()

Index(['budget', 'genres', 'original_language', 'popularity',
       'production_companies', 'release_date', 'runtime'],
      dtype='object')

In [259]:
#mean_value =
#trainingData[''].fillna(value=mean_value, inplace=True)

### Converting string to int

In [260]:
features.head(10)
features['original_language'].value_counts()

langauges = ['en', 'ja', 'es', 'no', 'fr', 'ko', 'zh', 'sv', 'cn', 'ru', 'id']
languageId = [x for x in range(len(langauges))]

features.replace(to_replace=langauges, value=languageId)




,budget,genres,original_language,popularity,production_companies,release_date,runtime
0,14000000,"[{'id': 35, 'name': 'Comedy'}]",0,6.575393,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",2/20/15,93.0
1,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",0,8.248895,"[{'name': 'Walt Disney Pictures', 'id': 2}]",8/6/04,113.0
2,3300000,"[{'id': 18, 'name': 'Drama'}]",0,64.299990,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...",10/10/14,105.0
3,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",hi,3.174936,NaN,3/9/12,122.0
4,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",5,1.148070,NaN,2/5/09,118.0
...,...,...,...,...,...,...,...
2995,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",0,9.853270,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",4/22/94,102.0
2996,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",7,3.727996,"[{'name': 'Memfis Film', 'id': 321}, {'name': ...",3/28/13,102.0
2997,65000000,"[{'id': 80, 'name': 'Crime'}, {'id': 28, 'name...",0,14.482345,"[{'name': 'New Line Cinema', 'id': 12}, {'name...",10/11/96,120.0
2998,42000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",0,15.725542,"[{'name': 'Jersey Films', 'id': 216}, {'name':...",1/16/04,90.0


# Visualization

In [261]:
def convertProductComp(field):
    string = field
    listOfId = []
    field = field.replace("'", '"')
    listOfDict = json.loads(field)
    for dict in listOfDict:
        listOfId.append(dict['id'])
    return listOfId

In [262]:


features.replace(to_replace=langauges, value=languageId)

,budget,genres,original_language,popularity,production_companies,release_date,runtime
0,14000000,"[{'id': 35, 'name': 'Comedy'}]",0,6.575393,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",2/20/15,93.0
1,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",0,8.248895,"[{'name': 'Walt Disney Pictures', 'id': 2}]",8/6/04,113.0
2,3300000,"[{'id': 18, 'name': 'Drama'}]",0,64.299990,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...",10/10/14,105.0
3,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",hi,3.174936,NaN,3/9/12,122.0
4,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",5,1.148070,NaN,2/5/09,118.0
...,...,...,...,...,...,...,...
2995,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",0,9.853270,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",4/22/94,102.0
2996,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",7,3.727996,"[{'name': 'Memfis Film', 'id': 321}, {'name': ...",3/28/13,102.0
2997,65000000,"[{'id': 80, 'name': 'Crime'}, {'id': 28, 'name...",0,14.482345,"[{'name': 'New Line Cinema', 'id': 12}, {'name...",10/11/96,120.0
2998,42000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",0,15.725542,"[{'name': 'Jersey Films', 'id': 216}, {'name':...",1/16/04,90.0


In [263]:
# Dette fungerer ikke!
features.dropna(subset=['production_companies'], inplace=True)

before = []
after = []
for row in features:

    before.append(row['production_companies'])
    try:
        after.append(convertProductComp(row['production_companies']))
    except:
        features.drop(row['id'])
#    if type(row) == float(): print(row)
features.replace(before, after)

features.head(15)


TypeError: string indices must be integers, not 'str'

In [251]:
features

# ML

# Deploy